# Low level "raw" OpenAI API, no libraries
## First setup request data
- messages array with "chat history" (since models are stateless)
- tools array that describes what LLM can "ask for to be executed on its behalf"
- we can define the function with any "schema" and execute it anyway we want, be it local function, API call or another LLM
- basic headers

In [1]:
const configRq = await fetch('./config.json');
config = await configRq.json(); // polyglot global variables are without modifiers

const model = 'gpt-4o';
requestData = {
    model: model,
    messages: [{
        role: 'user',
        content: 'Hello, can you please calculate square root of 4786?'
    }],
    temperature: 0,
    max_tokens: 4000,
    // top_p: 1,
    // frequency_penalty: 0,
    // presence_penalty: 0,
    // seed: null
};

requestData.tools = [
    {
        type: 'function',
        function: {
            name: 'square_root',
            description: 'Calculate square root of a number',
            parameters: {
                type: 'object',
                properties: {
                    number: {
                        type: 'number',
                        description: 'Number to calculate square root of'
                    }
                },
                required: ['number']
            }
        }
    }
];

return requestData;

## HTTP call 1
- Make simple HTTP call (fetch) to OpenAI with above payload
- print out "raw" response

In [2]:
apiUrl = 'https://api.openai.com/v1/chat/completions';
headers = {
    'Content-Type': 'application/json',
    Authorization: `Bearer ${config.OpenAIAPIKey}`,
};

mainRequest = await fetch(apiUrl, {
    method: 'POST',
    headers: headers,
    body: JSON.stringify(requestData),
});

if (!mainRequest.ok) {
    throw new Error(`HTTP error! status: ${mainRequest.status}`);
}

mainResponse = await mainRequest.json();

return mainResponse;

## Model responds with:
- general metadata (msg id, type, dates, model...)
- choices (can be multiple); this is answers
- the choice is a tool call; LLM is asking **us** to execute a tool we described
- arguments of the function called *usually* follow the schema provided above

## Now prepare second "round" of payload
- add tool call request to "history" for model's reference (optional; works even without it, *usually* =))
- read arguments of tool call and prepare result
- add tool result into "history"/context
   - again, LLMs are stateless (as of 07/2024)
   - we need to provide complete "conversation" for it "to guess the best next token"

In [3]:
toolCalled = mainResponse?.choices?.[0]?.message?.tool_calls?.[0];
console.log(`Tool called: ${toolCalled?.function?.name}, arguments: ${toolCalled?.function?.arguments}, id: ${toolCalled?.id}`);

toolArguments = JSON.parse(toolCalled?.function?.arguments);

requestData.messages.push({
    role: 'assistant',
    content: null,
    tool_calls: [
        {
            id: toolCalled?.id,
            type: 'function',
            function: {
                name: toolCalled?.function?.name,
                arguments: toolCalled?.function?.arguments,
            },
        },
    ],
});

sqrt = Math.sqrt(toolArguments.number); // fun: return 71.987897 instead of truth; even more fun return -9

requestData.messages.push({
    role: 'tool',
    content: JSON.stringify(sqrt),
    tool_call_id: toolCalled?.id,
});

return requestData.messages;

Tool called: square_root, arguments: {"number":4786}, id: call_eezJULpXERLzWqaqg4xmIL3j

## Payload 2 review
- see above messages array that gets sent to model
- note that tool call ID is kept in sync, so that model can identify multiple and complex tool call chains

## HTTP call 2
- again, simple fetch, nothing smarter

In [4]:
try {
    mainRequest2 = await fetch(apiUrl, {
        method: 'POST',
        headers: headers,
        body: JSON.stringify(requestData),
    });
} catch (error) {
    console.error(error);
    return error;
}

if (!mainRequest2.ok) {
    throw new Error(`HTTP error! status: ${mainRequest2.status}, ${await mainRequest2.json()}`);
}

mainResponse2 = await mainRequest2.json();

return mainResponse2;

## Final result
- model reads the context and understands the tool was called with some result
- it **may** or may **not** decide to use that result in its own answer

## Tips or homeworks
- What happens if the tool lies?
- What happens if the tool lies *too much*?
- Tool is a great way to force LLM to return exact structured output
   - "Fake" tool call that is never returned to LLM and never passed to history is great "hack"
   - Basically we want an answer from LLM in JSON
   - We have even a schema to follow
   - But if we "just say" it, it usually doesn't work well
   - Instead, we can create a tool with schema and even force LLM to prefer tool calls
   - We retrieve the "arguments", but those are our "result"; we throw away this part from conversation